# Downloading GUNW products using ariaDownload.py

**Author**: Brett A. Buzzanga, David Bekaert - Jet Propulsion Laboratory

This notebook demonstrates how to use the productAPI.py command line tool to download Sentinel 1 ARIA Geocoded UNWrapped interferogram (**GUNW**) products.  A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97).

The **`ariaDownload.py`** program wraps around the NASA's ASF DAAC API and [Bulk Download Service](https://bulk-download.asf.alaska.edu/help). The ASF Bulk Download Service handles most of the heavy lifting of the data-download and will conveniently skip previously downloaded files, and re-download partially downloaded files.  
In this notebook, we will demonstrate **`ariaDownload.py`** functionality along track 4, which intersects the U.S. East Coast in southeastern Virginia.

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

<div class="alert alert-danger">
<b>Download</b>:     
    
- The ***jupyter notebook* does not allow for interactive entering of your user-name and password, use the *jupyter terminal* instead** with the same command for interactive use.
</div>

## Overview of the ariaDownload.py program

Running **`ariaDownload.py`** with no options, or with **`-h`**, will show the parameters options as well as some examples. At minimum, users need to specify a spatial constraint either as a track number or bounding box (can be a shapefile).

Let us explore what some of the other options are:

In [ ]:
!ariaDownload.py

## 1. Example of ariaDownload output options

The user has the option to **download**, **count**, and generate a **kmz** corresponding to return of their search criteria. Below we provide example for each of these over Track 4.

### Count the number of products

To get a count of the number of products, without downloading data, provide the **`--output`** option with the **`count`** argument. To get information on the exact product filenames also include the verbose option **`-v`**.

In [ ]:
!ariaDownload.py --track 4 --output count

### Generate list of virtual products from ASF S3 bucket (BETA)

To generate a textfile containing a list of product URLs from the ASF S3 bucket, without downloading data, provide the **`--output`** option with the **`url`** argument. To get information on the exact product filenames also include the verbose option **`-v`**. Extracting layers virtually by leveraging this list of URLs is currently only supported by systems with the following packages: Linux kernel >=4.3 and libnetcdf >=4.5 

In [ ]:
!ariaDownload.py --track 4 --output url

We can now have a look at the generated textfile, which contains the URLs of all standard products over the specified track. As there are a lot of products we will only have a look at the first 10.

In [ ]:
!head -n 10 products/download_products_4track_0.txt

### Generate KMZ of products

To generate a Google Earth kmz, without downloading data, provide the **`--output`** option with the **`kmz`** argument. In case you also want to get information on the exact product filenames you can enbale the verbose option by adding **`-v`**. The result is shown in **Fig. 1**.

In [ ]:
!ariaDownload.py --track 4 --output kmz

<img src="./support_docs/track_004.png" alt="track" width="700">
<center><b>Fig. 1</b> Image of products available along Track 4. </center>

### Download products

To download the products, simply omit the **`--output`** option or specify it with the **`download`** argument. Examples for downloading all data on track 4 include:

`ariaDownload.py -t 004`

`ariaDownload.py -t 004 -o download`


By default, the products will be downloaded into the *./products* folder. You can specify a different location using the **`-w`** option, such as:

`ariaDownload.py -t 004 -w /insarHome`

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

## 2. Spatial-temporal subsetting

The **`ariaDownload.py`** program has a number of options availble for subsetting the search of products in the spatial and temporal domain, including **bounding box**, **start/end date**, **aquisition geometry**, **temporal baseline**, or **interferogram** options. Below, using the count option for brevity, we demonstrate each of them.

### Subsetting by Bounding Box

Bounding box can either be specified as **SNWE** coordinates (a string with quotation) or by providing a **shapefile** (GeoJSON or ESRI shapefiles) to the **`--bbox`** option. 

An example based on subsetting using **SNWE** coordinates and the corresponding kmz is shown in **Fig. 2**:

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count

<img src="./support_docs/crop_hr.png" alt="crop" width="700">

<center><b>Fig. 2</b> Image of products available along within bounding box 36.75S, 37.225N, -76.655W, -75.928E</center>

An example based on subsetting using a shape file:

In [ ]:
!ariaDownload.py --track 4 --bbox './support_docs/HR_North.shp' -o count

<img src="./support_docs/crop_shp.png" alt="crop" width="700">

<center><b>Fig. 3</b> Image of products available along within shapefile (blue shading). </center>

### Subsetting by start/end date

Subsetting in time can be done by specifying a range within which products need to fall. This is controlled using the **`--start YYYYMMDD` ** and **`--end YYYYMMDD`** options. By default the complete observational record is considered.

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --start 20190101 

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count -s 20190101  --end 20190401 --verbose

Note the use of the **`--verbose`** option to show the products found.

### Subsetting based on product temporal baseline

The **`--daysless`** and **`--daysmore`** options allow us to subset based on the temporal baseline, i.e. the time length (in days) between the reference and secondary SAR acquisitions of the interferogram. Note that `--daysless 12` and `--daysmore 12` specifies only pairs with temporal baselines smaller and larger than 12 (so not exactly separated by 12) are downloaded, respectively.

To select only interferograms products with temporal baselines shorter than 13 days:

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysless 13 -v

Select only pairs with temporal baselines, greater than a year:

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysmore 364

Select only pairs with moderate temporal baselines between two and six months:

In [ ]:
!ariaDownload.py --track 4  --bbox "36.75 37.225 -76.655 -75.928"  -o count -m 59 -l 181 -v 

### Subsetting based on interferogram pair

To find a specific interferogram combination one can use the **`--ifg YYYYMMDD_YYYYMMDD`** option.

In [ ]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  -o count --ifg "20161018_20160702" -v 